In [7]:
import os
import warnings

import cartopy.crs as ccrs
import matplotlib.pyplot as plt


import pyart
from pyart.testing import get_test_data

warnings.filterwarnings('ignore')

## Reading in Data Using Py-ART

When reading in a radar file, we use the `pyart.io.read` module.

`pyart.io.read` can read a variety of different radar formats, such as Cf/Radial, LASSEN, and more. 
The documentation on what formats can be read by Py-ART can be found here:

* [Py-ART IO Documentation](https://arm-doe.github.io/pyart/API/generated/pyart.io.html)

For most file formats listed on the page, using `pyart.io.read` should suffice since Py-ART has the ability to automatically detect the file format.

Let's use a sample data file from `pyart` - which is [**cfradial** format](https://github.com/NCAR/CfRadial).

When we read this in, we get a [`pyart.Radar` object](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.html#pyart.core.Radar)!

In [8]:
file = get_test_data('swx_20120520_0641.nc')
radar = pyart.io.read(file)
radar

### Investigate the [`pyart.Radar` object](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.html#pyart.core.Radar)

Within this [`pyart.Radar` object](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.html#pyart.core.Radar) object are the actual data fields.

This is where data such as reflectivity and velocity are stored.

To see what fields are present we can add the fields and keys additions to the variable where the radar object is stored.

In [9]:
radar.fields.keys()

dict_keys(['corrected_reflectivity_horizontal', 'reflectivity_horizontal', 'recalculated_diff_phase', 'specific_attenuation', 'unf_dp_phase_shift', 'mean_doppler_velocity', 'diff_phase', 'rain_rate_A', 'norm_coherent_power', 'dp_phase_shift', 'diff_reflectivity', 'proc_dp_phase_shift', 'copol_coeff'])

#### Extract a sample data field

The fields are stored in a dictionary, each containing coordinates, units and more.
All can be accessed by just adding the fields addition to the radar object variable.

For an individual field, we add a string in brackets after the fields addition to see
the contents of that field.

Let's take a look at `'corrected_reflectivity_horizontal'`, which is a common field to investigate.

In [10]:
print(radar.fields['corrected_reflectivity_horizontal'])

{'_FillValue': -9999.0, 'least_significant_digit': 2, 'units': 'dBZ', 'long_name': 'equivalent_reflectivity_factor', 'valid_min': -45.0, 'valid_max': 80.0, 'standard_name': 'equivalent_reflectivity_factor', 'data': masked_array(
  data=[[-5.671875, 2.28125, -8.1171875, ..., --, -13.4765625, --],
        [-5.6171875, 1.8984375, -10.0703125, ..., -2.6796875, -1.5390625,
         --],
        [-5.0390625, 2.625, -11.484375, ..., -8.984375, --, --],
        ...,
        [-5.9375, 1.46875, -12.3203125, ..., --, --, --],
        [-5.9609375, 1.53125, -12.84375, ..., --, --, --],
        [-8.7890625, 2.9140625, -12.09375, ..., --, --, --]],
  mask=[[False, False, False, ...,  True, False,  True],
        [False, False, False, ..., False, False,  True],
        [False, False, False, ..., False,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  

We can go even further in the dictionary and access the actual reflectivity data.

We use add `'data'` at the end, which will extract the **data array** (which is a masked numpy array) from the dictionary.

In [11]:
reflectivity = radar.fields['corrected_reflectivity_horizontal']['data']
print(type(reflectivity), reflectivity)

<class 'numpy.ma.core.MaskedArray'> [[-5.671875 2.28125 -8.1171875 ... -- -13.4765625 --]
 [-5.6171875 1.8984375 -10.0703125 ... -2.6796875 -1.5390625 --]
 [-5.0390625 2.625 -11.484375 ... -8.984375 -- --]
 ...
 [-5.9375 1.46875 -12.3203125 ... -- -- --]
 [-5.9609375 1.53125 -12.84375 ... -- -- --]
 [-8.7890625 2.9140625 -12.09375 ... -- -- --]]


Lets' check the size of this array

In [ ]:
reflectivity.shape

This reflectivity data array, numpy array, is a two-dimensional array with dimensions:
- Gates (number of samples away from the radar)
- Rays (direction around the radar)

In [ ]:
print(radar.nrays, radar.ngates)

If we wanted to look the 300th ray, at the second gate, we would use something like the following:

In [ ]:
print(reflectivity[300, 2])

## Plotting our Radar Data

### An Overview of Py-ART Plotting Utilities

Now that we have loaded the data and inspected it, the next logical thing to do is to visualize the data! Py-ART's visualization functionality is done through the objects in the [pyart.graph](https://arm-doe.github.io/pyart/API/generated/pyart.graph.html) module.

In Py-ART there are 4 primary visualization classes in pyart.graph:

* [RadarDisplay](https://meteoswiss.github.io/pyart/API/generated/pyart.graph.RadarDisplay.html)
* [RadarMapDisplay](https://meteoswiss.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.html)
* [AirborneRadarDisplay](https://meteoswiss.github.io/pyart/API/generated/pyart.graph.AirborneRadarDisplay.html)

Plotting grid data
* [GridMapDisplay](https://meteoswiss.github.io/pyart/API/generated/pyart.graph.GridMapDisplay.html)

### Use the [RadarMapDisplay](https://meteoswiss.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.html) with our data

For the this example, we will be using `RadarMapDisplay`, using Cartopy to deal with geographic coordinates.


We start by creating a figure first.

In [ ]:
fig = plt.figure(figsize=[10, 10])

Once we have a figure, let's add our `RadarMapDisplay`

In [ ]:
fig = plt.figure(figsize=[10, 10])
display = pyart.graph.RadarMapDisplay(radar)

Adding our map display without specifying a field to plot **won't do anything** we need to specifically add a field to field using `.plot_ppi_map()`

In [ ]:
display.plot_ppi_map('corrected_reflectivity_horizontal')

By default, it will plot the elevation scan, the the default colormap from `Matplotlib`... let's customize!

We add the following arguements:
- `sweep=3` - The fourth elevation scan (since we are using Python indexing)
- `vmin=-20` - Minimum value for our plotted field/colorbar
- `vmax=60` - Maximum value for our plotted field/colorbar
- `projection=ccrs.PlateCarree()` - Cartopy latitude/longitude coordinate system
- `cmap='pyart_HomeyerRainbow'` - Colormap to use, selecting one provided by PyART 

In [ ]:
fig = plt.figure(figsize=[12, 12])
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map('corrected_reflectivity_horizontal',
                     sweep=3,
                     vmin=-20,
                     vmax=60,
                     projection=ccrs.PlateCarree(),
                     cmap='pyart_HomeyerRainbow')
plt.show()

You can change many parameters in the graph by changing the arguments to plot_ppi_map. As you can recall from earlier. simply view these arguments in a Jupyter notebook by typing:

In [ ]:
display.plot_ppi_map?

For example, let's change the colormap to something different

In [ ]:
fig = plt.figure(figsize=[12, 12])
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map('corrected_reflectivity_horizontal',
                     sweep=3,
                     vmin=-20,
                     vmax=60,
                     projection=ccrs.PlateCarree(),
                     cmap='pyart_Carbone42')
plt.show()

Let's take a look at a different field - for example, correlation coefficient (`corr_coeff`)

In [ ]:
fig = plt.figure(figsize=[12, 12])
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map('copol_coeff',
                     sweep=0,
                     vmin=0.8,
                     vmax=1.,
                     projection=ccrs.PlateCarree(),
                     cmap='pyart_Carbone42')
plt.show()

---
## Summary
Within this notebook, we covered the basics of working with radar data using `pyart`, including:
- Reading in a file using `pyart.io`
- Investigating the `Radar` object
- Visualizing radar data using the `RadarMapDisplay`

### What's Next
In the next few notebooks, we walk through gridding radar data, applying data cleaning methods, and advanced visualization methods!

## Resources and References
Py-ART essentials links:

* [Landing page](https://meteoswiss.github.io/pyart/)
* [Examples](https://meteoswiss.github.io/pyart/examples/index.html)
* [Notebook Gallery](https://meteoswiss.github.io/pyart/notebook-gallery.html)
* [Source Code](https://github.com/MeteoSwiss/pyart)
* [Issue Tracker](https://github.com/MeteoSwiss/pyart/issues)